In [2]:
%matplotlib inline

from time import time
from math import sqrt, floor
import numpy as np
import pandas as pd
from IPython.core.debugger import set_trace
from pandas.plotting import scatter_matrix
pd.options.display.float_format = '{:.3f}'.format

import matplotlib.pyplot as plt
plt.style.use = "default"

In [3]:
def merge(df, labels):
    return labels.merge(df, left_index=True,right_index=True)

In [15]:
train = pd.read_csv("TestData4.txt", delim_whitespace=True, header=None)
labels = pd.read_csv("TrainLabel4.txt", sep='\t', header=None)
labels = labels.rename(columns={0: "label"})

In [16]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
count,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,...,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000,1092.000
mean,-0.096,-0.091,-0.103,-0.056,-0.033,-0.051,-0.043,-0.060,-0.036,0.100,...,-41.103,5.173,-0.770,-3.466,-22.087,13.747,41.928,-32.464,8.238,-9.480
std,0.073,0.071,0.068,0.037,0.029,0.033,0.101,0.104,0.092,0.064,...,164.829,175.244,177.987,180.498,166.496,155.011,155.689,159.812,172.023,175.198
min,-0.552,-0.347,-0.472,-0.372,-0.241,-0.317,-0.416,-0.415,-0.495,-0.011,...,-345.890,-273.350,-260.770,-250.690,-329.570,-325.260,-291.750,-308.070,-323.680,-283.870
25%,-0.113,-0.136,-0.127,-0.072,-0.038,-0.057,-0.113,-0.114,-0.079,0.052,...,-173.868,-172.562,-176.155,-184.202,-179.920,-160.312,-138.310,-179.495,-177.493,-176.507
50%,-0.073,-0.068,-0.088,-0.050,-0.024,-0.045,-0.019,-0.046,-0.019,0.079,...,-125.740,51.186,-88.748,17.669,-106.315,44.796,136.540,-123.610,89.193,-102.745
75%,-0.049,-0.036,-0.057,-0.029,-0.015,-0.030,0.000,0.012,0.029,0.135,...,148.088,180.132,182.328,180.097,175.127,168.105,175.460,158.832,177.303,178.222
max,-0.007,0.005,0.012,-0.004,-0.003,-0.007,0.379,0.130,0.228,0.355,...,319.090,320.420,271.100,332.260,327.620,288.520,302.370,338.210,303.600,255.210


In [18]:
# labels.value_counts()

In [19]:
# get columns with all values that have greater than 2 stds for the column.
stds = 2
above_stds_counts = train.apply(lambda x: x[x > np.abs(x.mean() + (x.std() * stds))].dropna().size).sort_values(ascending=False)
above_stds_counts = above_stds_counts[above_stds_counts > 0]
# above_stds_counts.head(40)

In [20]:
# for y in range(above_stds_counts.size):
#     h = train[above_stds_counts.index[y]]
#     h.plot.line()
#     h[h > np.abs(h.mean() + (h.std() * stds))].plot.line()
#     plt.title("columns: {} , index: {}".format(above_stds_counts.index[y], y))
#     plt.show()

Two standard deviations seems to get the best splits, and onlter after the first 40 std counds. Lets remove those

In [21]:
# train[above_stds_counts.index].hist(figsize=(18,15))

In [22]:
above_stds_counts = above_stds_counts.iloc[:40]
# for each col, get indices where std * 2 > mean.
f = train[above_stds_counts.index][train[above_stds_counts.index] > train.mean() + (train.std() * 2)].count(axis=1).sort_values(ascending=False)
train = train.drop(index=f[f.gt(5)].index)

In [9]:
train.describe()

,0,1,2,3,4,5,6,7,8,9,...,102,103,104,105,106,107,108,109,110,111
count,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,...,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000
mean,-0.090,-0.088,-0.098,-0.055,-0.032,-0.051,-0.051,-0.054,-0.035,0.096,...,-41.527,4.307,0.657,0.330,-26.696,14.662,42.814,-34.733,9.478,-10.601
std,0.061,0.068,0.061,0.036,0.028,0.031,0.091,0.098,0.091,0.061,...,164.623,175.106,177.997,180.675,165.753,154.305,155.233,159.175,172.064,174.856
min,-0.417,-0.347,-0.421,-0.285,-0.241,-0.246,-0.416,-0.415,-0.495,-0.011,...,-345.890,-273.350,-260.770,-250.690,-329.570,-325.260,-291.750,-308.070,-323.680,-283.870
25%,-0.109,-0.126,-0.123,-0.071,-0.038,-0.057,-0.113,-0.108,-0.077,0.051,...,-173.830,-172.470,-176.650,-184.380,-180.120,-159.460,-133.090,-179.780,-178.090,-175.960
50%,-0.072,-0.065,-0.086,-0.050,-0.023,-0.046,-0.021,-0.042,-0.018,0.077,...,-126.820,46.613,64.543,77.543,-111.260,46.337,136.390,-125.040,93.532,-106.280
75%,-0.049,-0.035,-0.056,-0.029,-0.015,-0.031,-0.002,0.014,0.028,0.128,...,147.330,180.280,182.280,180.290,175.240,167.630,175.930,155.750,177.380,178.600
max,-0.007,0.005,0.012,-0.004,-0.003,-0.007,0.379,0.130,0.228,0.355,...,319.090,320.420,271.100,332.260,327.620,288.520,302.370,338.210,303.600,255.210


In [23]:
# drop columns that are highly correlated with each other. 
corr = train.corr()
corr = corr[corr > .7]

In [24]:
def g(x):
    h = x[x.lt(1)].dropna()
    if not h.empty:
        return pd.Series([h.max(), h.idxmax()], index=["max", "column"])
    else:
        return np.nan
h = corr.apply(g)
h = h.T.dropna()

def j(x):
    smaller = h[h["column"] == x["column"]]["max"].idxmin()
    if x.name == smaller:
        return smaller
    else:
        return np.nan

h = h.drop(index=h[h.duplicated("column", False)].apply(j, axis=1).dropna())

dropped = []
def k(x):
    if x.name not in dropped:
        dropped.append(x.column)
        return x.column
    return np.nan

cols_to_drop = h.apply(k, axis=1).dropna()
train = train.drop(columns=cols_to_drop)

In [12]:
train.describe()

,0,2,3,4,5,6,7,8,9,11,...,93,94,95,98,99,100,101,102,103,104
count,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,...,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000,1045.000
mean,-0.090,-0.098,-0.055,-0.032,-0.051,-0.051,-0.054,-0.035,0.096,0.103,...,-55.401,8.652,24.788,0.228,-22.265,-8.741,57.458,-41.527,4.307,0.657
std,0.061,0.061,0.036,0.028,0.031,0.091,0.098,0.091,0.061,0.052,...,162.921,165.547,163.941,170.315,143.425,162.777,180.374,164.623,175.106,177.997
min,-0.417,-0.421,-0.285,-0.241,-0.246,-0.416,-0.415,-0.495,-0.011,0.006,...,-332.030,-302.700,-250.220,-213.240,-295.320,-281.000,-343.080,-345.890,-273.350,-260.770
25%,-0.109,-0.123,-0.071,-0.038,-0.057,-0.113,-0.108,-0.077,0.051,0.063,...,-180.000,-157.180,-168.540,-170.540,-150.560,-167.770,-154.970,-173.830,-172.470,-176.650
50%,-0.072,-0.086,-0.050,-0.023,-0.046,-0.021,-0.042,-0.018,0.077,0.097,...,-161.050,21.445,114.030,-92.163,-84.371,-53.213,152.150,-126.820,46.613,64.543
75%,-0.049,-0.056,-0.029,-0.015,-0.031,-0.002,0.014,0.028,0.128,0.135,...,143.160,179.430,178.210,177.740,138.090,170.000,197.980,147.330,180.280,182.280
max,-0.007,0.012,-0.004,-0.003,-0.007,0.379,0.130,0.228,0.355,0.360,...,320.180,336.570,333.020,254.850,249.470,237.350,314.090,319.090,320.420,271.100


In [25]:
g = merge(train, labels)
labels = g.pop("label")
labels.to_csv("cleaned_labels.csv")
train.to_csv("cleaned_test.csv")

In [14]:
train

,0,2,3,4,5,6,7,8,9,11,...,93,94,95,98,99,100,101,102,103,104
6,-0.227,-0.227,-0.030,-0.017,-0.019,0.174,-0.208,-0.090,0.182,0.257,...,184.990,-201.840,197.640,-176.340,175.130,-148.470,-204.820,176.280,-170.850,174.130
10,-0.344,-0.301,-0.029,-0.020,-0.016,0.166,-0.217,-0.077,0.199,0.220,...,173.870,-190.720,191.310,-176.400,-132.770,118.500,-151.900,161.700,-166.810,178.830
13,-0.158,-0.143,-0.017,-0.010,-0.009,0.176,-0.193,-0.086,0.098,0.134,...,182.380,-192.290,189.000,179.200,-161.280,176.010,164.520,-167.650,-182.190,176.520
14,-0.184,-0.173,-0.024,-0.017,-0.012,0.175,-0.199,-0.086,0.123,0.158,...,173.480,-190.570,198.850,-186.350,-146.910,159.900,159.890,-132.910,144.380,-174.380
16,-0.269,-0.150,-0.027,-0.024,-0.013,0.176,-0.199,-0.079,0.140,0.165,...,182.720,-194.850,192.480,-189.530,-165.250,198.130,152.790,-158.100,159.600,-175.180
18,-0.098,-0.091,-0.015,-0.007,-0.007,0.182,-0.189,-0.070,0.059,0.083,...,178.260,-185.680,188.060,-182.010,-165.810,172.930,172.080,-162.650,165.540,-176.080
20,-0.160,-0.108,-0.020,-0.020,-0.009,0.188,-0.179,-0.062,0.138,0.137,...,182.790,-193.310,191.790,-188.450,187.560,-150.340,144.150,-159.410,161.160,-173.380
21,-0.158,-0.113,-0.021,-0.014,-0.011,0.189,-0.176,-0.063,0.140,0.134,...,186.790,-195.680,190.920,-187.200,182.610,-139.970,138.600,-165.850,172.960,-179.640
22,-0.197,-0.164,-0.026,-0.025,-0.013,0.192,-0.166,-0.064,0.126,0.165,...,174.250,-187.760,190.530,-186.870,-156.440,175.160,-189.630,188.980,158.660,-166.240
25,-0.242,-0.179,-0.034,-0.038,-0.018,0.197,-0.153,-0.068,0.174,0.220,...,186.140,-200.400,197.530,165.880,173.450,-116.060,129.690,-142.900,132.190,-163.550
